# Ask PubMed and PubMed Central

In [1]:
import xml.etree.ElementTree as et 
import entrezpy.conduit
import datetime
import time
import sys
import os

normal = sys.stdout

In [2]:
boolean = True 

In [3]:
c = entrezpy.conduit.Conduit('email')

In [4]:
today = datetime.date.today()
day = today.day
month = today.month
year = today.year
date = str(day) + '_' + str(month) + '_' + str(year)
print(date)
os.makedirs(date, exist_ok=True)
os.chdir(date)

8_10_2022


# Recherche globale : 

## <span style='color:red'> Query 1 : </span> Pubmed : 

$\rightarrow$ nextflow and snakemake

### Requete

In [5]:
name1 = 'q1_' + date + ".xml"
if boolean:
    res = open(name1,'w')
    sys.stdout = res 
    extract_article = c.new_pipeline()
    sid = extract_article.add_search({'db' : 'pubmed',
                              'term' : 'nextflow or snakemake',
                              'rettype' : 'uilist'})

    fid = extract_article.add_fetch({'retmode' : 'xml','rettype' : 'abstract' }, dependency=sid)
    pipeline = c.run(extract_article)

    sys.stdout = normal
    res.close()
    time.sleep(3)

### Analyse

In [6]:
print(os.getcwd())
file1 = et.parse(name1)
pubmedXml = file1.getroot()
articles = list(pubmedXml)

/home/clemence/FAC/MASTER/M1/Stage/BDD_article/both/8_10_2022


In [7]:
print("Nombre de résultats avec la première requete : {}".format(len(articles)))

Nombre de résultats avec la première requete : 163


Extraction du titre, de l'abstract et des keywords pour chaque article trouvé et de l'id Pubmed:

In [8]:
def extractText(part, sup=False):
    txt = ''
    if part.tag != 'fig' and part.tag != 'table' and part.tag != 'table-wrap':  
        
        if part.tag == 'list-item':
            for subPart in part:
                if subPart.tag != 'label':
                    txt += '** ' + extractText(subPart) + '\n'
        else:
            if sup:
                txt += ' ['
                
            if part.text != None:
                if len(part) == 0:
                    txt += part.text
                else: 
                    txt += (part.text).strip() + " "

            for subPart in part:
                if subPart.tag =='xref':
                    txt = txt.strip() 
                if subPart.tag == 'ext-link':
                    txt = txt.strip() + " "
                    
                if subPart.tag == 'sup':
                    txt = txt.strip()
                    txt += extractText(subPart, True)
                else:
                    txt += extractText(subPart)

            if sup:
                txt += ']'
            if part.tail != None:
                if len(part.tail.strip()) != 0:
                    txt += part.tail
    
    return txt

In [9]:
idPubmed = []
abPubmedAll = {}
for article in articles:
    txt = ''
    medlineCitation = article.find('MedlineCitation')
    pmid = medlineCitation.find('PMID')
    idPubmed.append(pmid.text)
    
    art = medlineCitation.find('Article')
    title = art.find('ArticleTitle')
    abstract = art.find('Abstract')
        
    titre = extractText(title)
    txt += titre + '\n\n'
    try:
        ab = extractText(abstract)
        txt += ab + '\n\n'
    except:
        None
        
    try:
        keyword = medlineCitation.find('KeywordList')
        for k in keyword:
            txt += extractText(k) + ' '
    except:
        None
    abPubmedAll.update({pmid.text:txt})

## <span style='color:red'> Query 2 : </span> Pubmed : 

$\rightarrow$ ((nextflow[Title/Abstract]) OR (snakemake[Title/Abstract])) AND (github[Title/Abstract])

In [10]:
name2 = 'q2_' + date + ".xml"
if boolean:
    res = open(name2,'w')
    sys.stdout = res 
    extract_article = c.new_pipeline()
    sid = extract_article.add_search({'db' : 'pubmed',
                              'term' : '((nextflow[Title/Abstract]) OR (snakemake[Title/Abstract])) AND (github[Title/Abstract])',
                              'rettype' : 'uilist'})

    fid = extract_article.add_fetch({'retmode' : 'xml','rettype' : 'abstract' }, dependency=sid)
    pipeline = c.run(extract_article)

    sys.stdout = normal
    res.close()
    time.sleep(3)

In [11]:
file2 = et.parse(name2)
pubmedXml = file2.getroot()
articles = list(pubmedXml)

In [12]:
print("Nombre de résultats avec la seconde requete : {}".format(len(articles)))

Nombre de résultats avec la seconde requete : 75


In [13]:
idPubmed2 = []
for article in articles:
    medlineCitation = article.find('MedlineCitation')
    pmid = medlineCitation.find('PMID')
    idPubmed2.append(pmid.text)

In [14]:
for idx in idPubmed2:
    if not idx in idPubmed:
        print('Strange')

## <span style='color:red'> Query 3 : </span> PMC : 

$\rightarrow$ (nextflow[Abstract] OR snakemake[Abstract]) AND github[All Fields] 

In [15]:
name3 = 'q3_' + date + ".xml"
if boolean:
    res = open(name3,'w')
    sys.stdout = res 
    extract_article = c.new_pipeline()
    sid = extract_article.add_search({'db' : 'pmc',
                              'term' : '(nextflow[Abstract] OR snakemake[Abstract]) AND github[All Fields]',
                              'rettype' : 'uilist'})

    fid = extract_article.add_fetch({'retmode' : 'xml','rettype' : 'abstract' }, dependency=sid)
    pipeline = c.run(extract_article)

    sys.stdout = normal
    res.close()
    time.sleep(3)

### Analyse

In [16]:
file3 = et.parse(name3)
pubmedXml = file3.getroot()
articles = list(pubmedXml)

In [17]:
print("Nombre de résultats avec la troisieme requete : {}".format(len(articles)))

Nombre de résultats avec la troisieme requete : 110


Extraction des Pubmed ID des différents PMC ID:

In [18]:
linkId = {}
noOk = 0
for article in articles :
    if article.tag == 'Reply':
        print("Can't acces to this article : ",article.attrib)
        noOk +=1
    else:
        articleMeta = article.find("front/article-meta")
        ids = articleMeta.findall("article-id")
        temp = {}
        for idx in ids:
            temp[idx.attrib['pub-id-type']] = idx.text
            #print(idx.attrib, idx.text)
        try:
            pmc = 'PMC' + temp['pmc']
            linkId.update({pmc : temp['pmid']})
        except:
            print('This article have a problem PMC Id : {} -> no pmid'.format(temp['pmc']))
            noOk +=1

This article have a problem PMC Id : 9522874 -> no pmid
This article have a problem PMC Id : 7428130 -> no pmid


On regarde si les id trouvés dans la requete 3 sont inclus dans la requete 1:

In [19]:
nb = 0
for idx in linkId:
    pmid = linkId[idx]
    if not pmid in idPubmed:
        print(idx, linkId[idx])
        nb +=1
tot = len(articles)-noOk
print("\n{} sur {} de PMC ne se trouve pas dans mes resultats de la requete 1 sur PUBMED".format(nb,tot))

PMC8796378 34788790

1 sur 108 de PMC ne se trouve pas dans mes resultats de la requete 1 sur PUBMED


On regarde si les id trouvés dans la requete 3 sont inclus dans la requete 2:

In [20]:
nb = 0
for idx in linkId:
    pmid = linkId[idx]
    if not pmid in idPubmed2:
        #print(idx, linkId[idx])
        nb +=1
tot = len(articles)-noOk
print("\n{} sur {} de PMC ne se trouve pas dans mes resultats de la requete 2 sur PUBMED".format(nb,tot))


48 sur 108 de PMC ne se trouve pas dans mes resultats de la requete 2 sur PUBMED
